This notebook trains only the classification layer of the ResNet50 while keep the backbone frozon.

# Mount with GDrive + connect to GPU


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Dec  9 20:50:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Import Packages

In [ ]:
%%capture
# %%capture suppress the output of following !pip as they are tedious
!pip3 install pyyaml==5.1 pycocotools>=2.0.1 ipython
!pip3 install torch torchvision tensorflow 
!python3 -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip3 install Pillow
!pip3 install pandas
!pip3 install scikit-learn funcy
!pip install awscli

# Check if tensorboard is installed, otherwise install it
!if [[ $(pip3 list | grep -c tensorboard) -eq 0 ]]; then pip3 install tensorboard; else echo "tensorboard already installed"; fi

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import copy
import numpy as np
from urllib import request
import pandas as pd
import os, json, shutil, cv2, random, datetime, torch, torchvision
from shutil import copyfile
import PIL
from PIL import Image
from IPython.display import Image as IImage
from IPython.display import display, HTML
from IPython.display import IFrame
import json, funcy
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
import torchvision.transforms as transforms
import torch

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import Visualizer
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.modeling import build_model
from detectron2.utils.visualizer import ColorMode
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

import math

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [ ]:
print(PIL.PILLOW_VERSION)

8.0.1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: PILLOW_VERSION is deprecated and will be removed in a future release. Use __version__ instead.
  """Entry point for launching an IPython kernel.


In [ ]:
cd /content/

/content


In [ ]:
!git clone https://github.com/myc159/Deep-Taylor-Decomposition

Cloning into 'Deep-Taylor-Decomposition'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 101 (delta 26), reused 82 (delta 15), pack-reused 0
Receiving objects: 100% (101/101), 1.79 MiB | 616.00 KiB/s, done.
Resolving deltas: 100% (26/26), done.


In [ ]:
% cd Deep-Taylor-Decomposition/ 
# must go into Deep-Taylor-Decomposition/ so 'from model ... ' in the next cell can be executed

/content/Deep-Taylor-Decomposition


In [ ]:
# import argparse
import logging
import logging.handlers
import pdb

from model import *
from model import saliency_mapping as sa_map
from torch.autograd import Variable

from tqdm import tqdm 
from tqdm.auto import tqdm

import torch.nn as nn
from torchvision.utils import save_image
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.autograd import Variable

import requests

from sklearn import decomposition
from sklearn.preprocessing import MinMaxScaler

from collections import defaultdict

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 1
# random.seed(seed)
torch.manual_seed(seed)


import torch.nn.functional as F

from collections import OrderedDict
from model.resnet import BasicBlock, Bottleneck

try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url

/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


# Load Data

In [ ]:
civil_role_real_test_coco_path = "/content/drive/MyDrive/111 Rendered.ai/RarePlanes/datasets/coco_data/civil_role_real_test_coco.json"
civil_role_real_test_images_path = "/content/drive/MyDrive/111 Rendered.ai/RarePlanes/datasets/real/civil_test/"
role_model_dir = "/content/drive/MyDrive/111 Rendered.ai/RarePlanes/models/model_0023999.pth"
models_file = "/content/drive/MyDrive/111 Rendered.ai/RarePlanes/models/"

In [ ]:
# register data
register_coco_instances("civil_test", {}, civil_role_real_test_coco_path, civil_role_real_test_images_path)
satrgb_metadata = MetadataCatalog.get("civil_test")
test_dicts = DatasetCatalog.get("civil_test")

WARNING [12/09 20:50:49 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/09 20:50:49 d2.data.datasets.coco]: Loaded 2601 images in COCO format from /content/drive/MyDrive/111 Rendered.ai/RarePlanes/datasets/coco_data/civil_role_real_test_coco.json


In [ ]:
def load_coco_data(civil_role_real_test_coco_path):
    test_json_dir = civil_role_real_test_coco_path
    print("test json loc:", civil_role_real_test_coco_path)
    with open(test_json_dir) as json_file:
        civil_role_real_test_coco = json.load(json_file)
    print(civil_role_real_test_coco.keys())
    print("Num images in civil role real test         : {}".format(len(civil_role_real_test_coco['images'])))
    print("classes in civil role real test        : {}".format(civil_role_real_test_coco['categories']))
    print("Num bbox/instances in civil role real test : {}".format(len(civil_role_real_test_coco['annotations'])))
    return civil_role_real_test_coco

def load_model(role_model_dir):
    return torch.load(role_model_dir)
    print(role_model_pth['model']["roi_heads.box_predictor.cls_score.weight"].shape)
    print(role_model_pth['model']["roi_heads.box_predictor.bbox_pred.weight"].shape)

civil_role_real_test_coco = load_coco_data(civil_role_real_test_coco_path)
role_model = load_model(role_model_dir)

test json loc: /content/drive/MyDrive/111 Rendered.ai/RarePlanes/datasets/coco_data/civil_role_real_test_coco.json
dict_keys(['annotations', 'images', 'categories'])
Num images in civil role real test         : 2601
classes in civil role real test        : [{'id': 0, 'name': 'civil_small'}, {'id': 1, 'name': 'civil_medium'}, {'id': 2, 'name': 'civil_large'}]
Num bbox/instances in civil role real test : 6457


In [ ]:
torch.cuda.is_available()

True

# Load Pretrained Faster-RCNN

In [ ]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for bbox predication
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
# cfg.MODEL.WEIGHTS = "/content/gdrive/MyDrive/RarePlanes/models/model_0043999.pth" # faster rcnn for aircraft binary classification
cfg.MODEL.WEIGHTS = role_model_dir # faster rcnn for civil role classification
cfg.DATASETS.TEST = ("civil_test", )
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # 1 class for aircraft, 3 for civil role
predictor = DefaultPredictor(cfg)
fast_rcnn_model = build_model(cfg)

# Train ResNet 50 on predicted instances
Use predicted bbox and category_id from faster_cnn_model to finetune a ResNet50, because third-party DTD code only supports ResNet and ResNet is a backbone in Faster-RCNN. Extract the counterpart layers of ResNet from Faster-RCNN and load them into ResNet. Freeze these layers and only finetune the weights for last FC layer, because our ResNet50 only has 3 classes, not like 1000 classes from ImageNet

## Data loader

In [ ]:
# load cropped images and ann
cropped_imgs_dir = "/content/drive/MyDrive/111 Rendered.ai/data/cropped_civil_role_images/"
prediction_dict_df = pd.read_csv(cropped_imgs_dir + "instances_prediction_info.csv")

In [ ]:
# acc of un-finetined resnet50
train_dir = cropped_imgs_dir
batch_size = 1
num_workers = 4

train_dataset = datasets.ImageFolder(
                                      train_dir,
                                      transforms.Compose([
                                          transforms.Resize(224),
                                          transforms.CenterCrop(224),
                                          transforms.ToTensor(),
                                      ]))
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          num_workers=num_workers, pin_memory=True, shuffle=True,
                          drop_last=False)

## Load pretrained resnet 50 model whose weights = faster-rcnn

In [ ]:
def matching_fpn2res(faster_rnn, res50):
    fas2res = {"res2":"layer1", "res3":"layer2", "res4":"layer3", "res5":"layer4"}
    frcnn_dict = defaultdict(list)
    res50_dict = defaultdict(list)

    for fas_layer_i in fas2res:
        for k in faster_rnn["model"]:
          if fas_layer_i in k and "weight" in k and "shortcut" not in k:
            frcnn_dict[fas_layer_i].append(k)
        for k in res50.keys():
          res_layer_i = fas2res[fas_layer_i]
          if "weight" in k and res_layer_i in k and "down" not in k:
            res50_dict[res_layer_i].append(k)

    return frcnn_dict, res50_dict


def flatten_dict(dic):
    lst = []
    for k,v in dic.items():
        lst+=v
    return lst


def fpn2res_mapping(fas_lst, res50_lst):
    map = {}
    for i,k in enumerate(res50_lst):
        map[fas_lst[i]] = k
    return map

In [ ]:
# load pretrained faster_rcnn for weights extraction
faster_rcnn = role_model
# load pretrained resnet50
resnet_model = resnet50(pretrained=False, num_classes=3)
if not os.path.exists(models_file+"pretrain_res50.pth"):
    data = request.urlopen('https://download.pytorch.org/models/resnet50-19c8e357.pth', timeout=15).read()
    with open(models_file+"pretrain_res50.pth", 'wb') as f:
        f.write(data)
res50 = torch.load(models_file+"pretrain_res50.pth")

#modify the last FC layer shapes to cater for our 3 classes
res50['fc.weight'] = res50['fc.weight'][:3]
res50['fc.bias'] = res50['fc.bias'][:3]
# print(res50['fc.weight'].shape)
# print(res50['fc.bias'].shape)

# make model pth mapping
frcnn_dict, res50_dict = matching_fpn2res(faster_rcnn, res50)
fas_lst = flatten_dict(frcnn_dict)
res50_lst = flatten_dict(res50_dict)
mapping = fpn2res_mapping(fas_lst, res50_lst)

# match dictionary keys betweeen faster rcnn + resnet50
res50_by_fpn = res50
for k in mapping:
    res50_by_fpn[mapping[k]] = faster_rcnn["model"][k]
  
# load our resnet_model with extracted weights from pretrained faster-rcnn
resnet_model.load_state_dict(res50_by_fpn)
# freeze all but last 2 params of ResNet50
for i, param in enumerate(resnet_model.parameters()):
    param.requires_grad = True if i in [159,160] else False

## Training

In [ ]:
def train(model, train_loader, epoch, num_epoch, learning_rate, weight_decay, log_interval):
    # Using sum reduction simply bc otherwise, the scale of the loss is too small;
    # given that we always have 28*28 images, using sum/mean reduction wouldn't
    # make a difference.
    criteria = nn.CrossEntropyLoss(reduction='sum')
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate, weight_decay=weight_decay)

    print('model train start')
    model.train()
    total_loss = 0
    for i, data in enumerate(train_loader):
        print(f'i : {i}')
        images, targets = data
        targets -= 1

        # send to device
        if torch.cuda.is_available():
            images = images.to(device)
            targets = targets.to(device)
        
        # zero grad
        optimizer.zero_grad()
        # feed forward
        outputs = model(images)

        # print(targets.size)
        # print(outputs.size)

        # compute loss
        # for denoising version, loss is between dirty images output and clean input!
        loss = criteria(outputs, targets)
        # print(type(loss))
        total_loss += loss.item()

        # b
        loss.backward()
        optimizer.step()

        if (i + 1) % log_interval == 0:
            print('Epoch [{}/{}] - Iter[{}/{}], Cross Entropy loss:{:.6f}'.format(
                epoch + 1, num_epoch, i + 1,
                len(train_loader.dataset) // batch_size, loss.item() / batch_size
            ))

def validate(model, valid_loader, best_val_loss):
    total_loss = 0
    criteria = nn.CrossEntropyLoss(reduction='sum')
    model.eval()
    for i, data in enumerate(valid_loader):
        images, targets = data
        targets -= 1
        if torch.cuda.is_available():
            images = images.to(device)
            targets = targets.to(device)

        outputs = model(images)
        loss = criteria(outputs, targets)
        total_loss += loss
    
    avg_loss = total_loss / (len(valid_loader) * batch_size)

    print('\nLoss per batch on validation set: {:.6f}'.format(avg_loss))

    if avg_loss < best_val_loss:
        save_path = '/content/gdrive/MyDrive/cropped_civil_role_images/'
        best_val_loss = avg_loss
        # Save best model
        torch.save(model.state_dict(), save_path+"/model_"+str(best_val_loss))
        print('Saved best model in the checkpoint directory\n')
    
    return best_val_loss

def predict(model, train_loader):

    if torch.cuda.is_available():
        model = model.to(device)
    model.eval()

    correct = 0
    total = 0
    for i, data in tqdm(enumerate(train_loader)):

        # if i % 500 == 0:
        print(f'batch : {i + 1}')

        images, targets = data
        targets -= 1
        # print(targets)

        if torch.cuda.is_available():
            images = images.to(device)
            targets = targets.to(device)

        outputs = model(images)
        preds = torch.argmax(outputs, axis = 1) + 1

        # print(outputs)
        # print(preds)

        correct += (targets == preds).sum().item()
        total += targets.size(0)
    
    print('Accuracy : %d %%' % (100 * correct / total))

In [ ]:
# If want to further finetune resnet_model, de-comment this cell block
best_val_loss = float('inf')
num_epoch = 50
learning_rate = 1e-3
weight_decay = 0
log_interval = 10

if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"

device = torch.device(dev)  

resnet_model = resnet_model.to(dev)

for epoch in range(num_epoch):
    start_time = datetime.datetime.now()
    print('enter our_train() ...')
    train(resnet_model, train_loader, epoch, num_epoch, learning_rate, weight_decay, log_interval)
    end_time = datetime.datetime.now()
    print(f'Trained this epoch using {(end_time - start_time).seconds} seconds')
    # validate and save model if it's the best model so far
    best_val_loss = validate(resnet_model, train_loader, best_val_loss)
print('Training done; best validation loss {:.4f}'.format(best_val_loss))

enter our_train() ...
model train start
i : 0
i : 1
i : 2
i : 3
i : 4
i : 5
i : 6
i : 7
i : 8
i : 9
Epoch [1/50] - Iter[10/7844], Cross Entropy loss:0.557822
i : 10
i : 11
i : 12
i : 13
i : 14
i : 15
i : 16
i : 17
i : 18
i : 19
Epoch [1/50] - Iter[20/7844], Cross Entropy loss:0.007718
i : 20
i : 21
i : 22
i : 23
i : 24
i : 25
i : 26
i : 27
i : 28
i : 29
Epoch [1/50] - Iter[30/7844], Cross Entropy loss:1.955780
i : 30
i : 31
i : 32
i : 33
i : 34
i : 35
i : 36
i : 37
i : 38
i : 39
Epoch [1/50] - Iter[40/7844], Cross Entropy loss:0.058419
i : 40
i : 41
i : 42
i : 43
i : 44
i : 45
i : 46
i : 47
i : 48
i : 49
Epoch [1/50] - Iter[50/7844], Cross Entropy loss:6.043296
i : 50
i : 51
i : 52
i : 53
i : 54
i : 55
i : 56
i : 57
i : 58
i : 59
Epoch [1/50] - Iter[60/7844], Cross Entropy loss:0.005157
i : 60
i : 61
i : 62
i : 63
i : 64
i : 65
i : 66
i : 67
i : 68
i : 69
Epoch [1/50] - Iter[70/7844], Cross Entropy loss:0.265424
i : 70
i : 71
i : 72
i : 73
i : 74
i : 75
i : 76
i : 77
i : 78
i : 79
Epoc

KeyboardInterrupt: ignored

## Loaded well_trained resnet50

In [ ]:
# finetuned model is at /content/gdrive/MyDrive/cropped_civil_role_images/model_tensor(0.5385, device='cuda:0', grad_fn=<DivBackward0>)
resnet_model.load_state_dict(torch.load("/content/gdrive/MyDrive/cropped_civil_role_images/model_tensor(0.5385, device='cuda:0', grad_fn=<DivBackward0>)"))

<All keys matched successfully>